In [1]:
%load_ext autoreload

In [2]:
%autoreload 2

In [3]:
from theano import *
from lasagne.layers import EmbeddingLayer, InputLayer, get_output
import lasagne
import lasagne.layers
import theano.tensor as T
import theano
import numpy as np

In [37]:
l_in = lasagne.layers.InputLayer((10,50))

In [38]:
l_output = lasagne.layers.DenseLayer(l_in, num_units=1, name='output')

In [39]:
out = lasagne.layers.get_output(l_output)

In [40]:
print out

Elemwise{mul,no_inplace}.0


In [41]:
func = theano.function([l_in.input_var], out)

In [42]:
target_v = (np.random.randint(0,2,10) * 2 - 1).astype('int32')
target_t = np.random.randn(10, 50)

In [43]:
target_v.shape, target_t.shape

((10,), (10, 50))

In [44]:
func(target_t)

array([[ 0.        ],
       [ 0.23094998],
       [ 0.27202438],
       [ 0.        ],
       [ 0.15665717],
       [ 0.22074143],
       [ 1.20307148],
       [ 0.09498141],
       [ 0.        ],
       [ 0.83265301]])

In [45]:
all_params = lasagne.layers.get_all_params(l_output)

In [79]:
y_value = T.ivector('y')
loss = (y_value - out.reshape((out.size,))) ** 2

There was some issue with the shape of the ouput vector being [10,1] and the shape of the Y values being [10]

this hack above is computing the loss by transforming the output vector into a single dimentional item, and then we compute the difference between these two terms and square it

maybe simply taking the transpose of the y_value matrix can make these two values compatiable

In [81]:
update = lasagne.updates.adagrad(loss.mean(), all_params)

In [88]:
train = theano.function([l_in.input_var, y_value], (out, y_value, loss, loss.mean()), updates=update)

In [93]:
train(target_t, target_v)

[array([[  0.00000000e+00],
        [ -6.66133815e-16],
        [ -6.66133815e-16],
        [  2.22044605e-16],
        [  2.22044605e-16],
        [  0.00000000e+00],
        [ -2.22044605e-16],
        [  0.00000000e+00],
        [  2.22044605e-16],
        [  2.22044605e-16]]),
 array([ 1, -1, -1,  1,  1,  1,  1,  1,  1,  1], dtype=int32),
 array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.]),
 array(0.9999999999999997)]

In [69]:
out_v.shape

(10, 1)

In [70]:
y_v.shape

(10,)

In [76]:
out_v.size

10

In [100]:
T.matrix(y_v)

TypeError: ('name must be a string', array([ 1, -1, -1,  1,  1,  1,  1,  1,  1,  1], dtype=int32))